# YOLO - Training
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jul 2021<br>
**Descripción:** Notebook de entrenamiento.<br>

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
```

## Instalar zip

``` bash
    sudo apt install zip unzip
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/training`.

``` bash
    python3 -m venv yolo_training --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_training/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

* Agregar kernel a JupyterLab (**con el entorno virtual activado**).
    
``` bash
    python -m ipykernel install --user --name=yolo_training 
```

## Clonación y construcción de Darknet (por única vez)

In [8]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
Updating files:  37% (758/2044)
Updating files:  38% (777/2044)
Updating files:  39% (798/2044)
Updating files:  40% (818/2044)
Updating files:  41% (839/2044)
Updating files:  42% (859/2044)
Updating files:  43% (879/2044)
Updating files:  44% (900/2044)
Updating files:  45% (920/2044)
Updating files:  46% (941/2044)
Updating files:  47% (961/2044)
Updating files:  48% (982/2044)
Updating files:  49% (1002/2044)
Updating files:  50% (1022/2044)
Updating files:  51% (1043/2044)
Updating files:  52% (1063/2044)
Updating files:  53% (1084/2044)
Updating files:  54% (1104/2044)
Updating files:  55% (1125/2044)
Updating files:  56% (1145/2044)
Updating files:  57% (1166/2044)
Updating files:  57% (1171/2044)
Updating files:  58% (1186/2044)
Updating files:  59% (1206/2044)
Updating files:  60% (1227/2044)
Updating files:  61% (1247/2044)
Updating files:  62% (1268/2044)
Updating files:  63% (1288/2044)
Updating files:  64% (1309/2044)
Updating files:  65% (1329/20

In [9]:
%cd darknet

C:\Users\santi\Desktop\UALI\Repos\ml_uali-main\ml_uali\training\yolo_v2\darknet\darknet


### Localmente

In [10]:
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make -d

"make" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


### Server

Copiar Makefile_mod a Darknet/ modificando nombre a Makefile (solo en el server)

## Definir funciones auxiliares

In [11]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Entrenamiento personalizado
### Descargando conjunto de datos por URL

**Visdrone**

obj_data URL: https://drive.google.com/file/d/11XROrjMn6efEj19PKoxijcP0syivJIGP/view?usp=sharing

test_data URL: https://drive.google.com/file/d/1pCHS5HP4qd--EBWLjjqluzVbp-Ma6Q1Y/view?usp=sharing

readme URL: https://drive.google.com/file/d/1cubZwBp4p_keMsZc6Ne-LgDWZsuYKxrR/view?usp=sharing

In [13]:
%cd /content

[WinError 2] El sistema no puede encontrar el archivo especificado: '/content'
C:\Users\santi\Desktop\UALI\Repos\ml_uali-main\ml_uali\training\yolo_v2\darknet\darknet


In [14]:
# obj_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11XROrjMn6efEj19PKoxijcP0syivJIGP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11XROrjMn6efEj19PKoxijcP0syivJIGP" -O obj.zip && rm -rf /tmp/cookies.txt
# test_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pCHS5HP4qd--EBWLjjqluzVbp-Ma6Q1Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pCHS5HP4qd--EBWLjjqluzVbp-Ma6Q1Y" -O test.zip && rm -rf /tmp/cookies.txt

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [15]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pCHS5HP4qd--EBWLjjqluzVbp-Ma6Q1Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pCHS5HP4qd--EBWLjjqluzVbp-Ma6Q1Y" -O test.zip && rm -rf /tmp/cookies.txt

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [16]:
# descomprima los conjuntos de datos y su contenido para que ahora estén en la carpeta /darknet/data/
!unzip obj.zip -d darknet/data/
!unzip test.zip -d darknet/data/

# eliminando carpetas
!rm -rf obj.zip
!rm -rf test.zip

unzip:  cannot find or open obj.zip, obj.zip.zip or obj.zip.ZIP.
unzip:  cannot find or open test.zip, test.zip.zip or test.zip.ZIP.


### Copiando archivos de configuración


In [19]:
cp data_training/obj.data darknet/data/obj.data
cp data_training/obj.names darknet/data/obj.names
cp data_training/yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

SyntaxError: invalid syntax (<ipython-input-19-edec4dcd7c3b>, line 1)

### Descargando archivos de configuración (Solo en Colab)


In [ ]:
# descargando obj.data, obj.names y yolov4-obj.cfg 
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/obj.data
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/obj.names
!wget https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training/yolov4-obj.cfg

In [ ]:
!cp obj.data darknet/data/obj.data
!cp obj.names darknet/data/obj.names
!cp yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

In [ ]:
!rm -rf obj.data
!rm -rf obj.names
!rm -rf yolov4-obj.cfg

### Generando train.txt y test.txt

In [ ]:
%cd /content/darknet/

In [ ]:
import os

In [ ]:
# Generando test.txt

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [ ]:
# Generando train.txt

image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

### Descargue pesos pre-entrenados para las capas convolucionales.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### Entrena tu detector personalizado!

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg pre_training/yolov4.conv.137 -dont_show -map

In [ ]:
# muestre chart.png de cómo le fue al detector de objetos personalizado con la capacitación
imShow('chart.png')

 ### Iniciar el entrenamiento desde donde se guardó por última vez (hay que mirarlo!!!)

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_last.weights -dont_show -map

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_final.weights